In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from tqdm import tqdm

from scipy.spatial import ConvexHull

import lsst.daf.butler as dafButler


from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, ManualInterval, AsinhStretch, MinMaxInterval, LogStretch


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from typing import TYPE_CHECKING, cast

import astropy.units as u
from astropy.coordinates import SkyCoord

plt.set_loglevel('WARNING')

from tqdm import tqdm
import pandas as pd

In [ ]:
# Load collection
repo = '/repo/main'
instrument = "LSSTComCam"

butler_11= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472', instrument=instrument)
butler_12= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/d_2025_03_25/DM-49693', instrument=instrument)
butler_13= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751', instrument=instrument)
butler_14= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864', instrument=instrument)
butler = dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/v29_0_0_rc6/DM-50098', instrument=instrument)

In [ ]:
obj_refs = butler.query_datasets("object")

print(len(obj_refs))

In [ ]:
# List of tract indices to process
tract_indices = np.arange(0, len(obj_refs))

# Initialize lists to hold diaObj and diaSource DataFrames
Obj_list = []


# Loop through the specified tracts and fetch the data
for idx in tqdm(tract_indices):
    obj_ref = obj_refs[idx]  # Select the corresponding obj_ref
    Object_tract = butler.get(obj_ref)  # Fetch diaObj for this tract 
    Object_tract = Object_tract.to_pandas()  # Convert to DataFrame   
    # Append the results to the lists
    Obj_list.append(Object_tract)

# Concatenate all diaObj and diaSource DataFrames
combined_Obj = pd.concat(Obj_list, ignore_index=False)



In [ ]:
combined_Obj = combined_Obj[(combined_Obj['g_ra']>50) & (combined_Obj['g_ra']<63)]

In [ ]:
comcam_sne = pd.read_csv('comcam_sne.csv',names=['name', 'ra', 'dec'], header=None)

In [ ]:
plt.scatter(combined_Obj['g_ra'], combined_Obj['g_dec'])
plt.scatter(comcam_sne['ra'], comcam_sne['dec'], color='red', marker='x')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.title('Combined Object Data with SNe, field 1')
plt.xlim(51.5, 54)
plt.ylim(-29.1, -27)
plt.grid()
plt.show()

In [ ]:
plt.scatter(combined_Obj['g_ra'], combined_Obj['g_dec'])
plt.scatter(comcam_sne['ra'], comcam_sne['dec'], color='red', marker='x')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.title('Combined Object Data with SNe, field 2')
plt.xlim(57.5, 61)
plt.ylim(-50.1, -47)
plt.grid()
plt.show()

In [ ]:
import lsdb

In [ ]:
sne_lsdb = lsdb.from_dataframe(comcam_sne, ra_col='ra', dec_col='dec')

In [ ]:
sne_lsdb

In [ ]:
combined_Obj.columns

In [ ]:
combined_Obj_thin = combined_Obj[['objectId','coord_ra', 'coord_dec']]
combined_Obj_thin = combined_Obj_thin.rename(columns={'coord_ra': 'ra', 'coord_dec': 'dec'})
combined_Obj_thin

In [ ]:
obj_lsdb = lsdb.from_dataframe(combined_Obj_thin)

In [ ]:
obj_lsdb

In [ ]:
sne_obj_xmatch = sne_lsdb.crossmatch(obj_lsdb, radius_arcsec=0.1, left)

In [ ]:
sne_lsdb.

In [ ]:
lsdb_obj = lsdb.crossmatch(comcam_sne, combined_Obj_thin, radius_arcsec=0.5)

In [ ]:
lsdb_obj_computed = lsdb_obj.compute()

In [ ]:
lsdb_obj_computed

In [ ]:
object_dp1_full = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0/object_collection', search_filter=cone_search)

In [ ]:
object_dp1_full.columns

In [ ]:
from lsdb.core.search import ConeSearch
import nested_pandas as npd
cone_search_f1 = ConeSearch(ra=58.5, dec=-48, radius_arcsec=5 * 3600)
cone_search_f2 = ConeSearch(ra=52.5, dec=-27.5, radius_arcsec=5 * 3600)


object_dp1_f1 = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0/object_collection', columns=['objectId', "coord_ra", "coord_dec", 'objectForcedSource'], search_filter=cone_search_f1)
object_dp1_f2 = lsdb.read_hats('/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0/object_collection', columns=['objectId', "coord_ra", "coord_dec", 'objectForcedSource'], search_filter=cone_search_f2)

In [ ]:
object_dp1_computed = object_dp1_f1.compute()
object_dp2_computed = object_dp1_f2.compute()

In [ ]:
object_combined = pd.concat([object_dp1_computed, object_dp2_computed], axis=0)

In [ ]:
plt.scatter(object_combined['coord_ra'], object_combined['coord_dec'])
plt.scatter(comcam_sne['ra'], comcam_sne['dec'], color='red', marker='x')
plt.scatter(lsdb_obj_computed['ra_left'], lsdb_obj_computed['dec_left'], color='orange', marker='x')

plt.xlabel('RA')
plt.ylabel('Dec')

In [ ]:
# Define band → color mapping
band_colors = {
    'u': 'blue',
    'g': 'green',
    'r': 'red',
    'i': 'orange',
    'z': 'purple',
    'y': 'brown'
}

from matplotlib.patches import Patch

first_lc = object_dp1_computed[object_dp1_computed['objectId']==591818593281249758].objectForcedSource.iloc[0]

# Map colors to each row based on band
colors = first_lc['band'].map(band_colors)

# Create the plot
plt.figure(figsize=(10, 5))
plt.scatter(first_lc['midpointMjdTai'], first_lc['psfDiffFlux'],
            c=colors, alpha=0.7, edgecolor='k')

plt.gca().invert_yaxis()  # Magnitudes: brighter = lower value
plt.xlabel('MJD (midpointMjdTai)')
plt.ylabel('psfDiffFlux')
plt.title('Light Curve Colored by Band')
plt.ylim(-2000, 2000)
plt.grid(True)

# Create manual legend
legend_handles = [Patch(color=color, label=band) for band, color in band_colors.items() if band in first_lc['band'].unique()]
plt.legend(handles=legend_handles, title='Band', loc='best')

plt.show()

In [ ]:
object_ids

In [ ]:
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np

object_ids = lsdb_obj_computed['objectId_right'].unique()

for object_id in object_ids:
    matching_rows = object_combined[object_combined['objectId'] == object_id]
    if matching_rows.empty:
        continue

    first_lc = matching_rows.objectForcedSource.iloc[0]
    if first_lc is None or first_lc.empty:
        continue

    combined_row = lsdb_obj_computed[lsdb_obj_computed['objectId_right'] == object_id]
    if combined_row.empty:
        continue

    name = combined_row['name_left'].values[0]
    ra = combined_row['ra_right'].values[0]
    dec = combined_row['dec_right'].values[0]

    # Compute symmetric y-limits around 0 using 95% range
    flux = first_lc['psfDiffFlux'].dropna()
    if flux.empty:
        continue
    limit = np.percentile(np.abs(flux), 97.5) + 100
    y_min, y_max = -limit, limit

    # Start plot
    plt.figure(figsize=(10, 5))

    # Plot each band with its color
    for band, color in band_colors.items():
        band_data = first_lc[first_lc['band'] == band]
        if band_data.empty:
            continue
        plt.errorbar(
            band_data['midpointMjdTai'],
            band_data['psfDiffFlux'],
            yerr=band_data['psfDiffFluxErr'],
            fmt='o',
            color=color,
            ecolor=color,
            elinewidth=2,
            capsize=2,
            alpha=0.8,
            markeredgecolor='k',
            label=band
        )

    plt.gca().invert_yaxis()
    plt.xlabel('MJD (midpointMjdTai)')
    plt.ylabel('psfDiffFlux')
    plt.title(f'Object: {name} | RA: {ra:.5f}, Dec: {dec:.5f}', fontsize=12)
    plt.ylim(y_min, y_max)
    plt.xlim(60622, 60658)
    plt.grid(True)
    plt.legend(title='Band', loc='best')
    plt.show()

In [ ]:
first_lc['psfDiffFluxErr']

In [ ]:
lsdb_obj_computed

In [ ]:
2024ahzn,60.36688333333333,-48.51589166666667
2024ahxh,58.45505416666666,-48.65107777777778
2025efr,60.21437083333333,-48.260730555555554
2024ahwc,53.85345833333333,-27.98288333333333
2024aeyj,59.5316,-47.92468888888889
2024ahzm,58.18194583333333,-48.630272222222224
2024ahtp,58.119229166666656,-48.79749722222222
2025efu,60.594891666666655,-48.792991666666666
2025egc,59.263574999999996,-48.349672222222225
2024ahxm,51.91752083333333,-27.946872222222222
2024adon,53.45996249999999,-28.82738611111111
2024ahxv,59.54062083333333,-49.43626666666666
2025egz,59.84871666666666,-48.44846666666666
2022vtm,53.383062499999994,-27.154602777777775
2024ahwn,52.685316666666665,-27.807183333333334
2025ego,57.94811666666666,-49.09705833333334
2024ahyq,52.90686249999999,-28.33369722222222
2024ahyn,53.646245833333325,-28.655113888888888
2025egk,58.870787499999985,-48.92298888888889
2024ahuu,53.15265833333332,-28.298616666666668
2024ahzc,52.83825416666666,-28.279899999999998
2024ahst,58.52061666666666,-49.24156111111111
2024ahul,53.80612083333333,-27.7927
2024ahza,53.29867083333332,-28.87943611111111
2025egw,53.041675,-27.14548333333333
2025egl,59.85536249999999,-49.43301111111111
2025egs,53.548691666666656,-27.206097222222223
2024ahum,58.47711666666666,-49.577444444444446
2024ahuo,52.71615416666666,-28.990466666666666
2025egj,57.913404166666666,-48.445097222222216
2025ehg,52.58514583333333,-28.519927777777777
2025egq,53.95270416666666,-28.614255555555555


In [ ]:
2024ahul,53.80612083333333,-27.7927